In [ ]:
import pandas as pd
import CLdrawpic

# 函数划分

In [ ]:
def judge_kline_3_trend_type(pre_day,cur_day,last_day):
    if (cur_day[3] > pre_day[3]) and (cur_day[2] > pre_day[2]):
        if (last_day[3] > cur_day[3]) and (last_day[2] > cur_day[2]):
            return 1
        elif (last_day[3] <= cur_day[3]) and (last_day[2] <= cur_day[2]):
            return 2
    elif (cur_day[3] <= pre_day[3]) and (cur_day[2] <= pre_day[2]):
        if (last_day[3] > cur_day[3]) and (last_day[2] > cur_day[2]):
            return 3
        elif (last_day[3] <= cur_day[3]) and (last_day[2] <= cur_day[2]):
            return 4
    else:
        return 0

def generate_orignal_kindle_data(tmp_data):
    original_date = []
    original_kline = []
    for index, row in tmp_data.iterrows():
        #original_date.append(str(row['date']))
        original_kline.append(list(row[['open', 'close', 'low', 'high']].values))
    return original_kline

def delete_shadowline(original_kline):
    #输出去除上下影线的数据noshadow_kline_data
    noshadow_kline_data = []
    for i in range(len(original_kline)):
        k_line_tmp = original_kline[i]
        day_open,day_close,day_low,day_high = k_line_tmp[0], k_line_tmp[1], k_line_tmp[2], k_line_tmp[3]
        if day_open <= day_close:
            ktmp = [day_low,day_high,day_low,day_high] #去除上影线下影线
        else:
            ktmp = [day_high,day_low,day_low,day_high] #去除上影线下影线
        noshadow_kline_data.append(ktmp)
    return noshadow_kline_data
def contain_draw_line(noshadow_kline_data):
    #构造新的包含关系列表
    contain_kline_data = []
    #基于包含关系列表构造每段K线的走势 0下 1上
    kline_trend = []
    #包含关系对应的原列表index
    contain_index = []
    
    for i in range(len(noshadow_kline_data)):
        if i == 0 :
            contain_kline_data.append(noshadow_kline_data[i])
            kline_trend.append(1)
            contain_index.append(i)
        elif i >= 1:
            k_line_tmp_day = noshadow_kline_data[i]
            #获取今天开高收底
            day_open,day_close,day_low,day_high = k_line_tmp_day[0], k_line_tmp_day[1], k_line_tmp_day[2], k_line_tmp_day[3]
            k_line_tmp_lastday = contain_kline_data[-1]
            #获取昨天开高收底
            lastday_open,lastday_close,lastday_low,lastday_high  = k_line_tmp_lastday[0], k_line_tmp_lastday[1], k_line_tmp_lastday[2], k_line_tmp_lastday[3]
            #获取昨天相对前天的趋势
            last_trend = kline_trend[-1]
            
            #判断昨天今天状态（1上升、2今包昨、3昨包今、4下降）：
            flag = 0
            if (day_high > lastday_high) and (day_low > lastday_low):
                flag = 1
            elif ((day_high > lastday_high) and (day_low <= lastday_low)) or ((day_high == lastday_high) and (day_low <= lastday_low)):
                flag = 2
            elif ((day_high == lastday_high) and (day_low > lastday_low)) or ((day_high < lastday_high) and (day_low >= lastday_low)):
                flag = 3
            elif ((day_high < lastday_high) and (day_low < lastday_low)) or ((day_high < lastday_low)):
                flag = 4        
            
            #基于不同情况构造今天数据
            if flag == 1:#上升趋势
                contain_kline_data.append(noshadow_kline_data[i])
                kline_trend.append(1)
                contain_index.append(i)
            
            elif flag == 4:#下降趋势
                contain_kline_data.append(noshadow_kline_data[i])
                kline_trend.append(0)
                contain_index.append(i)
            
            elif flag == 2:#今天包含昨天
                #构造新的高价低价
                if last_trend == 1:#昨天上升
                    new_high = day_high
                    new_low = lastday_low
                elif last_trend == 0: #昨天下降
                    new_high = lastday_high
                    new_low = day_low
                #基于今天判断阴阳线
                if day_close >= day_open:
                    new_open = new_low
                    new_close = new_high
                else:
                    new_open = new_high
                    new_close = new_low
                #生成新数据
                contain_kline_data.pop()
                contain_kline_data.append([new_open,new_close,new_low,new_high])
                contain_index.pop()
                contain_index.append(i)
                
            elif flag == 3:#昨天包含今天
                #构造新的高价低价
                if last_trend == 1:#昨天上升
                    new_high = lastday_high
                    new_low = day_low
                elif last_trend == 0: #昨天下降
                    new_high = day_high
                    new_low = lastday_low    
                
                #基于昨天判断阴阳线
                if lastday_close >= lastday_open:
                    new_open = new_low
                    new_close = new_high
                else:
                    new_open = new_high
                    new_close = new_low
                #生成新数据
                contain_kline_data.pop()
                contain_kline_data.append([new_open,new_close,new_low,new_high])
                contain_index.pop()
                contain_index.append(i)
    #存储连续3根K线的4种形态 1三连升 2顶 3底 4三连降 0开头结尾
    top_button = [0]
    for i in range(1,len(contain_kline_data)-1):
        pre_day = contain_kline_data[i-1]
        cur_day = contain_kline_data[i]
        last_day = contain_kline_data[i+1]
        top_button.append(judge_kline_3_trend_type(pre_day,cur_day,last_day))
    top_button.append(0)
    
    #----------画笔----------
    #生成笔的坐标列表
    current_flag = ''
    current_high = 0 #保存当前顶或者底的最高值
    current_low = 0 #保存当前顶或者底的最低值
    current_index = 0 #当前顶或者底的x坐标
    draw_line_data = [] #保存笔的坐标 [[x1,y1],[x2,y2]] 
    
    for i in range(len(top_button)):
        if top_button[i] == 2: #找到一个顶
            if draw_line_data == []: #最终结果里还没填数据
                draw_line_data.append([i,contain_kline_data[i][3]]) #将当前顶的数据填上
                #更新顶的数据
                current_flag = 'top' 
                current_high = contain_kline_data[i][3]
                current_low = contain_kline_data[i][2]
                current_index = i
            elif draw_line_data != []:#最终结果里已经有数据了
                if current_flag == 'top': #前一个趋势是顶，但现在找到了新的顶
                    if contain_kline_data[i][3] >= current_high: #新的顶比前一个顶高
                        current_high = contain_kline_data[i][3]
                        current_low = contain_kline_data[i][2]
                        current_index = i
                        draw_line_data.pop()
                        draw_line_data.append([current_index,current_high])
                elif current_flag == 'bottom': #前一个趋势是底，现在招到了新的顶
                    if (i - current_index >= 4) and (contain_kline_data[i][3] >= current_high):
                        current_high = contain_kline_data[i][3]
                        current_low = contain_kline_data[i][2]
                        current_index = i
                        current_flag = 'top'
                        draw_line_data.append([current_index,current_high])
                        
        elif top_button[i] == 3: #找到一个底
            if draw_line_data == []: #最终结果里还没填数据
                draw_line_data.append([i,contain_kline_data[i][2]]) #将当前底的数据填上
                #更新顶的数据
                current_flag = 'bottom' 
                current_high = contain_kline_data[i][3]
                current_low = contain_kline_data[i][2]
                current_index = i
            elif draw_line_data != []:#最终结果里已经有数据了
                if current_flag == 'bottom': #前一个趋势是底，但现在找到了新的底
                    if contain_kline_data[i][2] <= current_low: #新的底比前一个底低
                        current_high = contain_kline_data[i][3]
                        current_low = contain_kline_data[i][2]
                        current_index = i                    
                        draw_line_data.pop()
                        draw_line_data.append([current_index,current_low])
    
                elif current_flag == 'top': #前一个趋势是顶，现在招到了新的底
                    if (i - current_index >= 4) and (contain_kline_data[i][2] <= current_low): 
                        current_high = contain_kline_data[i][3]
                        current_low = contain_kline_data[i][2]
                        current_index = i
                        current_flag = 'bottom'
                        draw_line_data.append([current_index,current_low])
    #----------适配非包含关系下的k线图----------
    new_draw = []
    for each in draw_line_data:
        newx = contain_index[each[0]]
        newy = each[1]
        new_draw.append([newx,newy])
    return new_draw

def cl_base(raw_day_data):
    original_kline = generate_orignal_kindle_data(raw_day_data)
    noshadow_line_data = delete_shadowline(original_kline)
    draw_1min = contain_draw_line(noshadow_line_data)
    return original_kline, draw_1min

def judge_seg(direction,curpeak,v0,v1,v2,v3,v4,v5,v6,v7,v8):
    if direction == 1:
        #标准顶分型
        if (v3 == curpeak) and (v3[1]>v1[1]) and (v2[1]>=v0[1]) and (v5[1]>=v2[1]):
            return 1
        #特征2包含特征3：
        elif (v3 == curpeak) and (v3[1]>v1[1]) and (v0[1]>v2[1]) and (v1[1]>=v4[1]):
            return 2
        #笔未被破坏但第三笔力度大
        elif (v3 == curpeak) and (v3[1]>v1[1]) and (v2[1]>v5[1]) and (v4[1]>=v0[1]):
            return 3
        #笔破坏且非常长
        elif (v3 == curpeak) and (v4[1]>v1[1]) and (v0[1]>=v2[1]):
            return 4
        #段破坏没封闭缺口但有足够长度
        elif (v5 == curpeak) and (v5[1]>v3[1]) and (v3[1]>=v1[1]) and (v2[1]>=v0[1]) and (v4[1]>=v2[1]) and (v4[1]>=v7[1]):
            return 5
        #段没破坏但形成新的特征序列
        elif (v6 == curpeak) and (v0[1]>v2[1]) and (v1[1]>v3[1]) and (v5[1]>v3[1]) and (v4[1]>v2[1]) and (v6[1]>v4[1]) and (v5[1]>v8[1]):
            return 6
        #连续三次没有冲破顶点
        elif (v6 == curpeak) and (v4[1]>v7[1]) and (v5[1]>=v3[1]) and (v6[1]>v4[1]) and (v6[1]>v2[1]) and (v6[1]>v0[1]):
            return 7

    elif direction == 0:
        if (v3 == curpeak) and (v3[1]<v1[1]) and (v2[1]<=v0[1]) and (v5[1]<=v2[1]):
            return 1
        elif (v3 == curpeak) and (v3[1]<v1[1]) and (v0[1]<v2[1]) and (v1[1]<=v4[1]):
            return 2
        elif (v3 == curpeak) and (v3[1]<v1[1]) and (v2[1]<v5[1]) and (v4[1]<=v0[1]):
            return 3
        elif (v3 == curpeak) and (v4[1]<v1[1]) and (v0[1]<=v2[1]):
            return 4
        elif (v5 == curpeak) and (v5[1]<v3[1]) and (v3[1]<=v1[1]) and (v2[1]<=v0[1]) and (v4[1]<=v2[1]) and (v4[1]<=v7[1]):
            return 5
        elif (v6 == curpeak) and (v0[1]<v2[1]) and (v1[1]<v3[1]) and (v5[1]<v3[1]) and (v4[1]<v2[1]) and (v6[1]<v4[1]) and (v5[1]<v8[1]):
            return 6
        elif (v6 == curpeak) and (v4[1]<v7[1]) and (v5[1]<=v3[1]) and (v6[1]<v4[1]) and (v6[1]<v2[1]) and (v6[1]<v0[1]):
            return 7


def generate_uplevelseg(input_draw):
    seg_uplevel_data = []
    seg_uplevel_data.append(input_draw[1])
    #段方向判断
    if (input_draw[2][1] > input_draw[0][1]):
        direction = 1
    elif (input_draw[2][1] <= input_draw[0][1]):
        direction = 0
    curpeak = input_draw[1]
    #笔方向判断
    if (input_draw[2][1] > input_draw[1][1]):
        segdirection = 1
    elif (input_draw[2][1] <= input_draw[1][1]):
        segdirection = 0
    
    i = 3
    while i <= (len(input_draw)-1):
        segdirection = 1 - segdirection
        if i < 9:
            v0 = input_draw[i]
            v1 = input_draw[i-1]
            v2 = input_draw[i-2]
            #当前是底，找顶
            if direction == 1:
                #当前是向上一笔
                if segdirection == 1:
                    #如果当前顶比前一个顶高：当前顶作为新顶
                    if v0[1] >= curpeak[1]:
                        curpeak = v0
                    #当前顶比前一个顶低：将当前peak加入列表，并转换方向
                    elif v0[1] < curpeak[1]:
                        seg_uplevel_data.append(curpeak)
                        curpeak = v1
                        direction = 0
                #当前是向下一笔
                elif segdirection == 0:
                    pass
            #当前是底，找顶
            if direction == 0:
                #当前是向上一笔
                if segdirection == 0:
                    #如果当前顶比前一个顶高：当前顶作为新顶
                    if v0[1] <= curpeak[1]:
                        curpeak = v0
                    #当前顶比前一个顶低：将当前peak加入列表，并转换方向
                    elif v0[1] > curpeak[1]:
                        seg_uplevel_data.append(curpeak)
                        curpeak = v1
                        direction = 1
                #当前是向下一笔
                elif segdirection == 1:
                    pass
        elif i >= 9:
            v0 = input_draw[i]
            v1 = input_draw[i-1]
            v2 = input_draw[i-2]
            v3 = input_draw[i-3]
            v4 = input_draw[i-4]
            v5 = input_draw[i-5]
            v6 = input_draw[i-6]
            v7 = input_draw[i-7]
            v8 = input_draw[i-8]
            
            
            #方向向上
            if direction == 1:
                res = judge_seg(1,curpeak,v0,v1,v2,v3,v4,v5,v6,v7,v8)
                #当前是向上一笔
                if segdirection == 1:
                    #判定特殊情况：段没破坏但形成新的特征序列
                    if (res == 6):
                        seg_uplevel_data.append(curpeak)
                        seg_uplevel_data.append(v3)
                        curpeak = v0
                    if (res == 7):
                        seg_uplevel_data.append(curpeak)
                        curpeak = v1
                        direction = 0
                    #如果当前顶比前一个顶高：当前顶作为新顶
                    elif v0[1] >= curpeak[1]:
                        curpeak = v0
                    #如果当前顶比前一个顶低：
                    elif v0[1] < curpeak[1]:
                        pass
                #当前是向下一笔
                elif segdirection == 0:
                    if (res == 1) or (res == 3) or (res == 5):
                        seg_uplevel_data.append(curpeak)
                        curpeak = v0
                        direction = 0
                    elif (res == 2):
                        tmp1 = input_draw[:i-2]
                        tmp2 = input_draw[i:]
                        input_draw = tmp1 + tmp2
                        i = i - 2
                    elif (res == 4):
                        seg_uplevel_data.append(v3)
                        curpeak = v0
                        direction = 0
    
            #方向向下
            elif direction == 0:
                res = judge_seg(0,curpeak,v0,v1,v2,v3,v4,v5,v6,v7,v8)
                if segdirection == 0:
                    if (res == 6):
                        seg_uplevel_data.append(curpeak)
                        seg_uplevel_data.append(v3)
                        curpeak = v0
                    if (res == 7):
                        seg_uplevel_data.append(curpeak)
                        curpeak = v1
                        direction = 1
                    elif v0[1] <= curpeak[1]:
                        curpeak = v0
                    elif v0[1] > curpeak[1]:
                        pass
                elif segdirection == 1:
                    if (res == 1) or (res == 3) or (res == 5):
                        seg_uplevel_data.append(curpeak)
                        curpeak = v0
                        direction = 1
                    elif (res == 2):
                        tmp1 = input_draw[:i-2]
                        tmp2 = input_draw[i:]
                        input_draw = tmp1 + tmp2
                        i = i - 2
                    elif (res == 4):
                        seg_uplevel_data.append(v3)
                        curpeak = v0
                        direction = 1
        i = i + 1
    if seg_uplevel_data[-1] != curpeak:
        seg_uplevel_data.append(curpeak)
    return seg_uplevel_data

In [ ]:
orignal_1min_data = pd.read_csv('300031.csv')
orignal_1min_data['code'] = '300031'
#orignal_1min_data['trade_times'] = orignal_1min_data['trade_time'].apply(lambda x: str(x)[-8:])  # 提取时间部分
orignal_1min_data['trade_times'] = orignal_1min_data['trade_time'].apply(lambda x: str(x).split(' ')[1])
#orignal_1min_data['trade_dates'] = orignal_1min_data['trade_time'].apply(lambda x: str(x)[:10])  # 提取时间部分
orignal_1min_data['trade_dates'] = orignal_1min_data['trade_time'].apply(lambda x: str(x).split(' ')[0])
orignal_1min_data.columns = ['datetime','open','high','low','close','volume','amount','code','trade_times','trade_dates']
orignal_1min_data['datetime'] = pd.to_datetime(orignal_1min_data['trade_dates'].astype(str) + ' ' + orignal_1min_data['trade_times'])
orignal_1min_data.set_index('datetime', inplace=True)

In [ ]:
original_kline, L1seg = cl_base(orignal_1min_data)
L2seg = generate_uplevelseg(L1seg)

In [ ]:
kindlechart2 = CLdrawpic.generate_kindle_chart(original_kline,'sh688428')
drawL1_chart2 = CLdrawpic.generate_line_chart(L1seg,'L1seg','black')
drawL2_chart2 = CLdrawpic.generate_line_chart(L2seg,'L2seg','red')

kindlechart2.overlap(drawL1_chart2)
kindlechart2.overlap(drawL2_chart2)

kindlechart2.load_javascript()

In [ ]:
kindlechart2.render_notebook()

In [ ]:
def generate_zhongshu_position(start_x,start_y,end_x,end_y):
    tmp_list = []
    #构造起点坐标
    tmp_position_start = {'xAxis': 0, 'yAxis': 0}
    tmp_position_start['xAxis'] = start_x
    tmp_position_start['yAxis'] = start_y
    tmp_list.append(tmp_position_start)

    #构造终点坐标
    tmp_position_end = {'xAxis': 0, 'yAxis': 0}
    tmp_position_end['xAxis'] = end_x
    tmp_position_end['yAxis'] = end_y
    tmp_list.append(tmp_position_end)
    #print(tmp_list)
    #存储完成的中枢坐标
    return tmp_list

In [ ]:
draw_line_data = L2seg
mark_areas = []
zhongshu_start_x = 0
zhongshu_start_y = 0
zhongshu_end_x = 0
zhongshu_end_y = 0
zhongshu_influence_flag = 0


for i in range(1,len(draw_line_data)):
    if i == 1:
        zhongshu_start_x = draw_line_data[0][0]
        zhongshu_end_x = draw_line_data[1][0]
        zhongshu_start_y = max(draw_line_data[0][1],draw_line_data[1][1])
        zhongshu_end_y = min(draw_line_data[0][1],draw_line_data[1][1])
        zhongshu_influence_flag = zhongshu_influence_flag + 1

    elif i > 1:
        #如果此线段为上升
        if draw_line_data[i][1] > draw_line_data[i-1][1]:
            #则看该线段的高点相对中枢的位置
            #如果比中枢高点高，则只调整中枢结束坐标X轴，同时中枢影响线段+1
            if draw_line_data[i][1] >= zhongshu_start_y:
                zhongshu_end_x = draw_line_data[i][0]
                zhongshu_influence_flag = zhongshu_influence_flag + 1
            #如果比中枢高点低，低点高，则下调高点,同时调整中枢结束坐标X轴，同时中枢影响线段+1
            elif (draw_line_data[i][1] < zhongshu_start_y) and (draw_line_data[i][1] >= zhongshu_end_y):
                zhongshu_start_y = draw_line_data[i][1]
                zhongshu_end_x = draw_line_data[i][0]
                zhongshu_influence_flag = zhongshu_influence_flag + 1
            #如果比中枢低点更低，则此中枢结束，
            elif (draw_line_data[i][1] < zhongshu_end_y):
                #判断该中枢是否影响超过5个线段
                #如果是，则将该中枢加入存储列表，并初始化中枢影响线段数据
                if zhongshu_influence_flag >= 4:
                    mark_areas.append(generate_zhongshu_position(zhongshu_start_x,zhongshu_start_y,zhongshu_end_x,zhongshu_end_y))
                    zhongshu_influence_flag = 1
                #如果不是，则不形成中枢，直接将中枢影响标志置为1
                elif zhongshu_influence_flag < 4:
                    zhongshu_influence_flag = 1
                #不管前面中枢是否被加为列表， 此中枢都已结束，需初始化起点终点坐标
                zhongshu_start_x = draw_line_data[i-1][0]
                zhongshu_end_x = draw_line_data[i][0]
                zhongshu_start_y = max(draw_line_data[i-1][1],draw_line_data[i][1])
                zhongshu_end_y = min(draw_line_data[i-1][1],draw_line_data[i][1])
                    
        #如果此线段为下降
        elif draw_line_data[i][1] < draw_line_data[i-1][1]:
            #则看该线段的低相对中枢的位置
            #如果比中枢低点低，则只调整中枢结束坐标X轴，同时中枢影响线段+1
            if draw_line_data[i][1] <= zhongshu_end_y:
                zhongshu_end_x = draw_line_data[i][0]
                zhongshu_influence_flag = zhongshu_influence_flag + 1
            #如果比中枢高点低，低点高，则上调低点,同时调整中枢结束坐标X轴，同时中枢影响线段+1
            elif (draw_line_data[i][1] < zhongshu_start_y) and (draw_line_data[i][1] >= zhongshu_end_y):
                zhongshu_end_y = draw_line_data[i][1]
                zhongshu_end_x = draw_line_data[i][0]
                zhongshu_influence_flag = zhongshu_influence_flag + 1
            #如果比中枢高点更高，则此中枢结束，
            elif (draw_line_data[i][1] > zhongshu_end_y):
                #判断该中枢是否影响超过5个线段
                #如果是，则将该中枢加入存储列表，并初始化中枢影响线段数据
                if zhongshu_influence_flag >= 4:
                    mark_areas.append(generate_zhongshu_position(zhongshu_start_x,zhongshu_start_y,zhongshu_end_x,zhongshu_end_y))
                    zhongshu_influence_flag = 1
                #如果不是，则不形成中枢，直接将中枢影响标志置为1
                elif zhongshu_influence_flag < 4:
                    zhongshu_influence_flag = 1
                #不管前面中枢是否被加为列表， 此中枢都已结束，需初始化起点终点坐标
                zhongshu_start_x = draw_line_data[i-1][0]
                zhongshu_end_x = draw_line_data[i][0]
                zhongshu_start_y = max(draw_line_data[i-1][1],draw_line_data[i][1])
                zhongshu_end_y = min(draw_line_data[i-1][1],draw_line_data[i][1])
      
mark_areas.append(generate_zhongshu_position(zhongshu_start_x,zhongshu_start_y,zhongshu_end_x,zhongshu_end_y))


In [ ]:
mark_areas

In [ ]:
kindlechart3 = CLdrawpic.generate_kindle_chart(original_kline,'sh688428')
drawL1_chart3 = CLdrawpic.generate_line_chart(L1seg,'L1seg','black')
drawL2_chart3 = CLdrawpic.generate_line_chart_square_area(L2seg,'L2seg','blue',mark_areas)

kindlechart3.overlap(drawL1_chart3)
kindlechart3.overlap(drawL2_chart3)

kindlechart3.load_javascript()

In [ ]:
kindlechart3.render_notebook()